In [9]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/1984.csv' 
df = pd.read_csv(csv_file_path)

columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

date_format = '%Y-%m-%d'  
df['datadate'] = pd.to_datetime(df['datadate'], format=date_format)
df.sort_values(by=['tic', 'datadate'], inplace=True)

df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) - 1
) * 100

df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days 
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

pivot_df = df.pivot_table(index='datadate', columns='tic', values='daily_return')

pivot_df = pivot_df.map(lambda x: x / 100 if pd.notnull(x) else np.nan)

adjusted_daily_returns = pivot_df.map(lambda x: x + 1 if pd.notnull(x) else np.nan)

def consecutive_zeros(prices):
    count = 0
    max_count = 0
    for price in prices:
        if price == 0:
            count += 1
            max_count = max(max_count, count) 
        else:
            count = 0
    return max_count

consecutive_zero_counts = df.groupby('tic')['prccd'].apply(consecutive_zeros)

for ticker, max_zeros in consecutive_zero_counts.items():
    if max_zeros > 5:
        warnings.warn(f"Warning: Ticker '{ticker}' has more than 5 consecutive zeros in 'prccd'.")

all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')  
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()  # Last Monday
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data.isnull().sum(axis=0)[ticker] == 5:
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                weekly_pivot_df.loc[monday, ticker] = (weekly_data[ticker].prod(skipna=True) - 1) * 100

pd.set_option('display.float_format', lambda x: f'{x:.2f}')  

weekly_pivot_path = "weekly_pivot_1984.csv"  
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))


Weekly Pivot Table saved to: weekly_pivot_1984.csv
Weekly Returns Pivot Table (first 10 rows):
tic         0223B  0485B  0491B  3234B  3614B  3ACDGE  3ACKH  3ACTRQ  3ADLN  \
1984-01-09   0.49 -10.29   3.52 -10.71   1.30    2.25  -1.30    3.51  -2.50   
1984-01-16   0.48  -3.28  -2.72  -0.67  -5.79    8.79  -3.95   -3.39  -5.13   
1984-01-23   0.00  -8.47  -0.70   1.34   2.73    2.02  -2.74   -7.89   0.00   
1984-01-30  -0.48  -7.41   2.11   0.00   1.33    2.96  -6.10   -3.81  -2.70   
1984-02-06   1.45 -10.00  -2.07  -4.64  -3.61   -8.65  -3.44   -6.93  -5.56   
1984-02-13  -1.07  -0.00   2.11   0.00   0.00   -5.27  -4.71   -7.45  -2.94   
1984-02-20  -0.00   2.22   1.38   1.39  -6.80    1.12   0.55   10.34   0.00   
1984-02-27  -0.49  -5.43  -4.76   9.59   8.39   -2.20   7.10   -4.69   6.06   
1984-03-05  -1.98 -19.54   2.86  -3.12  -3.37    0.00  -2.55   -6.56  -2.86   
1984-03-12   1.01  -2.86  -4.17  -2.27  -3.83   -4.49  10.99    0.88  -2.94   

tic         3ADOT.  ...   ZGCO   ZI

In [ ]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/2008.csv' 
df = pd.read_csv(csv_file_path)

columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

date_format = '%Y-%m-%d'  
df['datadate'] = pd.to_datetime(df['datadate'], format=date_format)
df.sort_values(by=['tic', 'datadate'], inplace=True)

df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) - 1
) * 100

df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days 
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

pivot_df = df.pivot_table(index='datadate', columns='tic', values='daily_return')

pivot_df = pivot_df.map(lambda x: x / 100 if pd.notnull(x) else np.nan)

adjusted_daily_returns = pivot_df.map(lambda x: x + 1 if pd.notnull(x) else np.nan)

def consecutive_zeros(prices):
    count = 0
    max_count = 0
    for price in prices:
        if price == 0:
            count += 1
            max_count = max(max_count, count) 
        else:
            count = 0
    return max_count

consecutive_zero_counts = df.groupby('tic')['prccd'].apply(consecutive_zeros)

for ticker, max_zeros in consecutive_zero_counts.items():
    if max_zeros > 5:
        warnings.warn(f"Warning: Ticker '{ticker}' has more than 5 consecutive zeros in 'prccd'.")

all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')  
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()  # Last Monday
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data.isnull().sum(axis=0)[ticker] == 5:
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                weekly_pivot_df.loc[monday, ticker] = (weekly_data[ticker].prod(skipna=True) - 1) * 100

pd.set_option('display.float_format', lambda x: f'{x:.2f}')  

weekly_pivot_path = "weekly_pivot_2008.csv"  
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))


In [ ]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/2009.csv' 
df = pd.read_csv(csv_file_path)

columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

date_format = '%Y-%m-%d'  
df['datadate'] = pd.to_datetime(df['datadate'], format=date_format)
df.sort_values(by=['tic', 'datadate'], inplace=True)

df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) - 1
) * 100

df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days 
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

pivot_df = df.pivot_table(index='datadate', columns='tic', values='daily_return')

pivot_df = pivot_df.map(lambda x: x / 100 if pd.notnull(x) else np.nan)

adjusted_daily_returns = pivot_df.map(lambda x: x + 1 if pd.notnull(x) else np.nan)

def consecutive_zeros(prices):
    count = 0
    max_count = 0
    for price in prices:
        if price == 0:
            count += 1
            max_count = max(max_count, count) 
        else:
            count = 0
    return max_count

consecutive_zero_counts = df.groupby('tic')['prccd'].apply(consecutive_zeros)

for ticker, max_zeros in consecutive_zero_counts.items():
    if max_zeros > 5:
        warnings.warn(f"Warning: Ticker '{ticker}' has more than 5 consecutive zeros in 'prccd'.")

all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')  
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()  # Last Monday
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data.isnull().sum(axis=0)[ticker] == 5:
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                weekly_pivot_df.loc[monday, ticker] = (weekly_data[ticker].prod(skipna=True) - 1) * 100

pd.set_option('display.float_format', lambda x: f'{x:.2f}')  

weekly_pivot_path = "weekly_pivot_2009.csv"  
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))


In [ ]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/2010.csv' 
df = pd.read_csv(csv_file_path)

columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

date_format = '%Y-%m-%d'  
df['datadate'] = pd.to_datetime(df['datadate'], format=date_format)
df.sort_values(by=['tic', 'datadate'], inplace=True)

df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) - 1
) * 100

df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days 
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

pivot_df = df.pivot_table(index='datadate', columns='tic', values='daily_return')

pivot_df = pivot_df.map(lambda x: x / 100 if pd.notnull(x) else np.nan)

adjusted_daily_returns = pivot_df.map(lambda x: x + 1 if pd.notnull(x) else np.nan)

def consecutive_zeros(prices):
    count = 0
    max_count = 0
    for price in prices:
        if price == 0:
            count += 1
            max_count = max(max_count, count) 
        else:
            count = 0
    return max_count

consecutive_zero_counts = df.groupby('tic')['prccd'].apply(consecutive_zeros)

for ticker, max_zeros in consecutive_zero_counts.items():
    if max_zeros > 5:
        warnings.warn(f"Warning: Ticker '{ticker}' has more than 5 consecutive zeros in 'prccd'.")

all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')  
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()  # Last Monday
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data.isnull().sum(axis=0)[ticker] == 5:
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                weekly_pivot_df.loc[monday, ticker] = (weekly_data[ticker].prod(skipna=True) - 1) * 100

pd.set_option('display.float_format', lambda x: f'{x:.2f}')  

weekly_pivot_path = "weekly_pivot_2010.csv"  
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))


In [ ]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/2011.csv' 
df = pd.read_csv(csv_file_path)

columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

date_format = '%Y-%m-%d'  
df['datadate'] = pd.to_datetime(df['datadate'], format=date_format)
df.sort_values(by=['tic', 'datadate'], inplace=True)

df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) - 1
) * 100

df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days 
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

pivot_df = df.pivot_table(index='datadate', columns='tic', values='daily_return')

pivot_df = pivot_df.map(lambda x: x / 100 if pd.notnull(x) else np.nan)

adjusted_daily_returns = pivot_df.map(lambda x: x + 1 if pd.notnull(x) else np.nan)

def consecutive_zeros(prices):
    count = 0
    max_count = 0
    for price in prices:
        if price == 0:
            count += 1
            max_count = max(max_count, count) 
        else:
            count = 0
    return max_count

consecutive_zero_counts = df.groupby('tic')['prccd'].apply(consecutive_zeros)

for ticker, max_zeros in consecutive_zero_counts.items():
    if max_zeros > 5:
        warnings.warn(f"Warning: Ticker '{ticker}' has more than 5 consecutive zeros in 'prccd'.")

all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')  
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()  # Last Monday
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data.isnull().sum(axis=0)[ticker] == 5:
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                weekly_pivot_df.loc[monday, ticker] = (weekly_data[ticker].prod(skipna=True) - 1) * 100

pd.set_option('display.float_format', lambda x: f'{x:.2f}')  

weekly_pivot_path = "weekly_pivot_2011.csv"  
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))


In [ ]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = '/Users/balmeru/Downloads/2012.csv' 
df = pd.read_csv(csv_file_path)

columns_to_check = ['ajexdi', 'prccd', 'trfd']
df.dropna(subset=columns_to_check, how='any', inplace=True)

date_format = '%Y-%m-%d'  
df['datadate'] = pd.to_datetime(df['datadate'], format=date_format)
df.sort_values(by=['tic', 'datadate'], inplace=True)

df['adjusted_close'] = df['prccd'] / df['ajexdi']
df['adjusted_close_prior'] = df.groupby('tic')['adjusted_close'].shift(1)
df['trfd_prior'] = df.groupby('tic')['trfd'].shift(1)

df['daily_return'] = (
    ((df['adjusted_close'] * df['trfd']) /
     (df['adjusted_close_prior'] * df['trfd_prior'])) - 1
) * 100

df['day_diff'] = df.groupby('tic')['datadate'].diff().dt.days 
df.loc[df['day_diff'] > 5, 'daily_return'] = np.nan

pivot_df = df.pivot_table(index='datadate', columns='tic', values='daily_return')

pivot_df = pivot_df.map(lambda x: x / 100 if pd.notnull(x) else np.nan)

adjusted_daily_returns = pivot_df.map(lambda x: x + 1 if pd.notnull(x) else np.nan)

def consecutive_zeros(prices):
    count = 0
    max_count = 0
    for price in prices:
        if price == 0:
            count += 1
            max_count = max(max_count, count) 
        else:
            count = 0
    return max_count

consecutive_zero_counts = df.groupby('tic')['prccd'].apply(consecutive_zeros)

for ticker, max_zeros in consecutive_zero_counts.items():
    if max_zeros > 5:
        warnings.warn(f"Warning: Ticker '{ticker}' has more than 5 consecutive zeros in 'prccd'.")

all_weekdays = pd.date_range(start=adjusted_daily_returns.index.min(), end=adjusted_daily_returns.index.max(), freq='B')  
df_reindexed = adjusted_daily_returns.reindex(all_weekdays)

all_mondays = pd.date_range(start=df_reindexed.index.min(), end=df_reindexed.index.max(), freq='W-MON')
weekly_pivot_df = pd.DataFrame(data=np.nan, index=all_mondays, columns=df_reindexed.columns)
ticker_start_dates = df_reindexed.apply(lambda x: x.first_valid_index(), axis=0)

for i, monday in enumerate(all_mondays):
    start_date = monday
    if i < len(all_mondays) - 1:
        end_date = all_mondays[i + 1] - pd.DateOffset(days=1)
    else:
        end_date = df_reindexed.index.max()  # Last Monday
    
    weekly_data = df_reindexed.loc[start_date:end_date]
    for ticker in df_reindexed.columns:
        if monday < ticker_start_dates[ticker]:
            weekly_pivot_df.loc[monday, ticker] = np.nan
        else:
            if weekly_data.isnull().sum(axis=0)[ticker] == 5:
                weekly_pivot_df.loc[monday, ticker] = np.nan
            else:
                weekly_pivot_df.loc[monday, ticker] = (weekly_data[ticker].prod(skipna=True) - 1) * 100

pd.set_option('display.float_format', lambda x: f'{x:.2f}')  

weekly_pivot_path = "weekly_pivot_2012.csv"  
weekly_pivot_df.to_csv(weekly_pivot_path)
print(f"Weekly Pivot Table saved to: {weekly_pivot_path}")
print("Weekly Returns Pivot Table (first 10 rows):")
print(weekly_pivot_df.head(10))
